In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/network-attack-dataset-kitsune/SSL Renegotiation/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Setup environment
!pip install pyspark
# !pip install sparkmagic


     |████████████████████████████████| 212.3 MB 13 kB/s s eta 0:00:01    |████████████                    | 80.0 MB 32.6 MB/s eta 0:00:05     |█████████████████████▌          | 142.9 MB 22.3 MB/s eta 0:00:04
     |████████████████████████████████| 198 kB 50.7 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=793d4d0720724ea14c72c5862da8a81707b8be3049a871a110c059e03a0fa4d9
  Stored in directory: /root/.cache/pip/wheels/43/47/42/bc413c760cf9d3f7b46ab7cd6590e8c47ebfd19a7386cd4a57
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id, shiftRight, when, expr, lit, broadcast, sum
from pyspark.sql.window import Window

In [ ]:
spark = SparkSession.builder.appName("outlier-detection").getOrCreate()

In [ ]:
# Load Dataset
ssl_reneg_dataset = "../input/network-attack-dataset-kitsune/SSL Renegotiation/SSL_Renegotiation_dataset.csv"
arp_spoof_dataset = "../input/network-attack-dataset-kitsune/SSL Renegotiation/SSL_Renegotiation_dataset.csv"
syn_dos_dataset = "../input/network-attack-dataset-kitsune/SSL Renegotiation/SSL_Renegotiation_dataset.csv"


df_ssl = spark.read.load(
    ssl_reneg_dataset,
    format="csv", sep=",", inferSchema="true", header="false").withColumn("category", lit(0))
df_arp = spark.read.load(
    arp_spoof_dataset,
    format="csv", sep=",", inferSchema="true", header="false").withColumn("category", lit(1))
df_syn = spark.read.load(
    syn_dos_dataset,
    format="csv", sep=",", inferSchema="true", header="false").withColumn("category", lit(2))


In [ ]:
# feature distrivution and analysis
#LIMITS
df_ssl = df_ssl.limit(10000)
# df_ssl.printSchema()
df_ssl.select("_c0","_c1", "_c3").describe().show()
df_arp = df_arp.limit(10000)
# df_arp.printSchema()
df_arp.select("_c0","_c1", "_c3").describe().show()
df_syn = df_syn.limit(10000)
# df_syn.printSchema()
df_syn.select("_c0","_c1", "_c3").describe().show()

In [ ]:
# Union
df = df_ssl.union(df_arp)
df = df.union(df_syn)
df.count()

**GENERATES ID**

In [ ]:
# Generate monotonous id
df_data = df.withColumn("row_id", monotonically_increasing_id())

# analyze spark internal partitions
df_partition = df_data.withColumn("partition_id", shiftRight('row_id', 33)) \
    .withColumn("row_offset", df_data['row_id'].bitwiseAND(2147483647))
partitions_size = df_partition.groupBy("partition_id").count().withColumnRenamed("count", "partition_size")
windowSpec = Window.orderBy("partition_id").rowsBetween(Window.unboundedPreceding, -1) # warnings for window without a partition

# Take care of the null partition_offsets
partitions_offset = partitions_size.withColumn("partition_offset", when(expr("partition_id = 0"), lit(0))
                                               .otherwise(sum("partition_size").over(windowSpec)))
df_data_id = df_partition.join(broadcast(partitions_offset), "partition_id").withColumn("id",partitions_offset.partition_offset + df_partition.row_offset + 1).drop("partition_id", "row_id", "row_offset", "partition_size", "partition_offset")

In [ ]:
#join with labels and filter only malign records

**K-MEANS CLUSTERING**

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.pipeline import Pipeline
from pyspark.sql import dataframe
from pyspark.sql.functions import col

In [ ]:
vecAssembler = VectorAssembler(
    inputCols=[x for x in df_data_id.columns if x != "label"],
    outputCol="features"
)

df_kmeans = vecAssembler.transform(df_data_id).select('id', 'features')
df_kmeans.show()

In [ ]:
import numpy as np
import pandas as pd
from pyspark.ml.clustering import KMeans

k_means = KMeans().setK(3).setSeed(1).setPredictionCol("cluster").setFeaturesCol("features")
model = k_means.fit(df_kmeans)
centers = model.clusterCenters()
# print("Centers: ",centers)




In [ ]:
tf_df = model.transform(df_kmeans)
# tf_df.select("id", "cluster").show()

In [ ]:
tf_df = tf_df.join(df_data_id, 'id')
tf_df.select("id", "_c0", "_c114", "cluster").show()

In [ ]:
# COnvet to pandas
pddf_pred = tf_df.toPandas().set_index('id')
pddf_pred.head()

In [ ]:
import matplotlib.pyplot as plt

threedee = plt.figure(figsize=(12,10)).gca(projection='3d')
threedee.scatter(pddf_pred._c0, pddf_pred._c1, pddf_pred._c114, c=pddf_pred.cluster)
threedee.set_xlabel('x')
threedee.set_ylabel('y')
threedee.set_zlabel('z')
plt.show()
